# Box Office Analysis

authors: Greg Fatouras and Evan Rosenbaum

# Business Problem

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

# Sources of Data 

- Box Office Mojo
    - CSV
    - https://www.boxofficemojo.com/
- IMDB
    - DB
    - https://www.imdb.com/
- Rotten Tomatoes
    - TSVs
    - https://www.rottentomatoes.com/
- The Movie DB
    - CSV
    - https://www.themoviedb.org/
- The Numbers
    - CSV
    - https://www.the-numbers.com/

In [46]:
# Import packages
import sqlite3
import zipfile
import pandas as pd

# IMDB extraction

In [47]:
# Extract db from zipped file
zip_path = 'zippedData/im.db.zip'
extract_dir = 'zippedData'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [48]:
# Connect to SQLite3
conn = sqlite3.connect('zippedData/im.db') 
cursor = conn.cursor()

# View tables query
tables_query = """
SELECT *
FROM sqlite_master
WHERE type='table'
"""

pd.read_sql(tables_query, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [49]:
# List each table
table_names = [
    'movie_basics'
    ,'directors'
    ,'known_for'
    ,'movie_akas'
    ,'movie_ratings'
    ,'persons'
    ,'principals'
    ,'writers'
]

# Extract the columns from each table in the database
for table in table_names:
    headers_query = f"SELECT * FROM {table}"
    df = pd.read_sql(headers_query, conn)
    print(f"\nHeader for {table}:")
    print(df.columns.to_list())



Header for movie_basics:
['movie_id', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres']

Header for directors:
['movie_id', 'person_id']

Header for known_for:
['person_id', 'movie_id']

Header for movie_akas:
['movie_id', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'is_original_title']

Header for movie_ratings:
['movie_id', 'averagerating', 'numvotes']

Header for persons:
['person_id', 'primary_name', 'birth_year', 'death_year', 'primary_profession']

Header for principals:
['movie_id', 'ordering', 'person_id', 'category', 'job', 'characters']

Header for writers:
['movie_id', 'person_id']


In [50]:

## Extract the movie basics table into a DataFrame
imdb_query = """
with movie_info as(
select *
from movie_basics
),

cast_director_writer_info as (
select 
    movie_id
    ,GROUP_CONCAT(DISTINCT CASE WHEN category in ('actor','actress') THEN primary_name END) AS actors_list
    ,GROUP_CONCAT(DISTINCT CASE WHEN category = 'director' THEN primary_name END) AS directors_list
    ,GROUP_CONCAT(DISTINCT CASE WHEN category = 'writer' AND job LIKE '%screenplay%' THEN primary_name END) AS screenwriters_list
from principals
inner join persons
on principals.person_id = persons.person_id
where category IN ('actor', 'actress', 'director', 'writer')
group by movie_id
),

movie_rating as (
select *
from movie_ratings
)

select 
    movie_info.movie_id
    ,primary_title
    ,start_year
    ,runtime_minutes
    ,genres
    ,actors_list
    ,directors_list
    ,screenwriters_list
    ,averagerating
    ,numvotes
from movie_info
left join cast_director_writer_info
on movie_info.movie_id = cast_director_writer_info.movie_id
left join movie_rating
on movie_info.movie_id = movie_rating.movie_id

"""
imdb_df = pd.read_sql(imdb_query, conn)
imdb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            146144 non-null  object 
 1   primary_title       146144 non-null  object 
 2   start_year          146144 non-null  int64  
 3   runtime_minutes     114405 non-null  float64
 4   genres              140736 non-null  object 
 5   actors_list         99391 non-null   object 
 6   directors_list      128436 non-null  object 
 7   screenwriters_list  7820 non-null    object 
 8   averagerating       73856 non-null   float64
 9   numvotes            73856 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 11.2+ MB


# RT, BOM, TMDB, and TN extraction

In [51]:
# Load all other dataframes and view columns
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
rt_movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', encoding='latin1', delimiter='\t')
rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', encoding='latin1', delimiter='\t')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tn_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', encoding='utf-8')

dfs = [bom_df ,rt_movie_info_df ,rt_reviews_df ,tmdb_df ,tn_df]
dfs_str = ['bom_df' ,'rt_movie_info_df' ,'rt_reviews_df' ,'tmdb_df' ,'tn_df']

for x in range(len(dfs)):
    print(dfs_str[x])
    print(dfs[x].columns)
    print("\n")

bom_df
Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')


rt_movie_info_df
Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')


rt_reviews_df
Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')


tmdb_df
Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')


tn_df
Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')




### Box Office Mojo Exploration

In [52]:
# View high-level information about the DataFrame
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


#### 'studio'

In [53]:
# View NaNs in the 'studio' column
bom_df[bom_df['studio'].isna()]

,title,studio,domestic_gross,foreign_gross,year
210,Outside the Law (Hors-la-loi),NaN,96900.0,3300000,2010
555,Fireflies in the Garden,NaN,70600.0,3300000,2011
933,Keith Lemon: The Film,NaN,NaN,4000000,2012
1862,Plot for Peace,NaN,7100.0,NaN,2014
2825,Secret Superstar,NaN,NaN,122000000,2017


Overview
    - It isn't clear as to why the studios for these films are not included inside of the BOM csv. 

Action
    - While the information exists for these five films, to maintain consistency with other tables we are going to enter the studio as 'unknown'. 

#### 'domestic_gross' and 'foreign_gross' 

In [54]:
# View NaNs in the 'foreign_gross' column
bom_df[bom_df['foreign_gross'].isna()]

,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018



- Overview
    - NaN values inside of these column indicate that there was no revenue generated from either a domestic or international release. The movie might have had a subsequent releases either domestic or international post their initial release, but the revenue from that is not captured inside of the row as it occurred in a different year. 

- Example
    - Flipped (2010)
        - https://www.boxofficemojo.com/title/tt0817177/?ref_=bo_se_r_1
        - The films original release had no international revenue generated, but upon further releases in 2017 and 2020 there was.
        - That said, since the revenue was generated in a different year, it is not tied to the record in question. 

- Action
    - Fill the NaN values inside of both the 'domestic_gross' and 'foreign_gross' columns with 0's.

In [55]:
# View transformed DataFrame
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### Rotten Tomatoes Exploration

In [56]:
rt_movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [57]:
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


Overview 
    - The rotten tomatoes data does not list the movie that it relates too. They have id columns which internally tie the tables together, but the data relates specifically to rotten tomatoes and not the other sources.

### The Movie DB Exploration

In [58]:
# View high-level information about the DataFrame
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [59]:
# View DataFrame results
tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


- Overview
    - There are no NaN values inside of the DataFrame so no actions are needed to ensure missingness is handled. 

- Notes
    - genre_ids
        - https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee
        - We can map the ids to be the specific genres rather than the id (if needed)


### The Numbers Exploration

In [60]:
tn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [61]:
tn_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


- Overview
    - There are no NaN values inside of the DataFrame so no actions are needed to ensure missingness is handled. 

### IMDB DB Exploration

In [62]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            146144 non-null  object 
 1   primary_title       146144 non-null  object 
 2   start_year          146144 non-null  int64  
 3   runtime_minutes     114405 non-null  float64
 4   genres              140736 non-null  object 
 5   actors_list         99391 non-null   object 
 6   directors_list      128436 non-null  object 
 7   screenwriters_list  7820 non-null    object 
 8   averagerating       73856 non-null   float64
 9   numvotes            73856 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 11.2+ MB


# Dataframe Cleaning


### bom_df

In [63]:
# Fill NaNs with 'unknown'
bom_df['studio'].fillna('unknown', inplace=True)
# Fill NaNs with 0's
bom_df[['domestic_gross', 'foreign_gross']] = bom_df[['domestic_gross', 'foreign_gross']].fillna(0)

# bom_df : Remove (3D) and years at the end of the title strings
bom_df['title'] = bom_df['title'].str.replace(' \(3D\)', '').str.replace(' \(\d{4}\)', '')

bom_df["title"].value_counts() # One duplicate
bom_df_clean = bom_df[bom_df["title"] != "Bluebeard"] # Remove bluebeard duplicate
bom_df_clean.isna().value_counts() # No nan's
bom_df = bom_df_clean
bom_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,0,2018
3383,Edward II (2018 re-release),FM,4800.0,0,2018
3384,El Pacto,Sony,2500.0,0,2018
3385,The Swan,Synergetic,2400.0,0,2018


### tmdb

In [64]:
# Drop duplicate ID's
tmdb_df = tmdb_df.drop_duplicates(subset=['id'])


# Find differing titles
different = tmdb_df['original_title'] != tmdb_df['title']

# Filter the DataFrame 
differences = tmdb_df[different]

# 'Original title' contains foreign language titles
differences

tmdb_df = tmdb_df.drop('original_title', axis = 1)

tmdb_df


,Unnamed: 0,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,0.600,2018-06-22,Trailer Made,0.0,1


### The Numbers

In [65]:
# Sort by date
tn_df = tn_df.sort_values(by = 'release_date', ascending = False)
# Keep the first duplicated value, drop the rest
tn_df = tn_df.drop_duplicates(subset='title', keep='first')
# Remove any titles that do not start with a normal char
tn_df = tn_df[tn_df['title'].str.match(r'^[A-Za-z]')]

tn_df




,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [21]:
# Viewing individual movies to see if the are duplicates or distinct
tn_movie_list = [
    'King Kong'
    ,'Home'
    ,'The Square'
    ,'Flatliners'
    ,'Around the World in 80 Days'
    ,'The Fog'
    ,'Point Break'
]

tn_filtered_df = tn_df[tn_df['movie'].isin(tn_movie_list)]

# Sort the DataFrame by id in ascending order
tn_filtered_df_sorted = tn_filtered_df.sort_values(by='movie')

tn_filtered_df_sorted

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
340,41,"Jun 16, 2004",Around the World in 80 Days,"$110,000,000","$24,004,159","$72,004,159"
4120,21,"Oct 17, 1956",Around the World in 80 Days,"$6,000,000","$42,000,000","$42,000,000"
2127,28,"Aug 10, 1990",Flatliners,"$26,000,000","$61,308,153","$61,308,153"
2610,11,"Sep 29, 2017",Flatliners,"$20,000,000","$16,883,115","$45,173,738"
243,44,"Mar 27, 2015",Home,"$130,000,000","$177,397,510","$385,997,896"
3455,56,"Jun 5, 2009",Home,"$12,000,000",$0,$0
5459,60,"Apr 23, 2009",Home,"$500,000","$15,433","$44,793,168"
39,40,"Dec 14, 2005",King Kong,"$207,000,000","$218,080,025","$550,517,357"
2374,75,"Dec 17, 1976",King Kong,"$23,000,000","$52,614,445","$90,614,445"
5396,97,"Apr 7, 1933",King Kong,"$672,000","$10,000,000","$10,000,650"


- Review
    - Since all of these movies are distinct, we are going to keep the duplicated movie titles that exist inside of this table. 

## Merged DF

In [22]:
# bom_df
bom_filtered_df = bom_df[['title', 'year', 'domestic_gross', 'foreign_gross']]

In [23]:

# tmdb_df
# Convert the date column to datetime
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'])
# Extract the year
tmdb_df['year'] = tmdb_df['release_date'].dt.year
# Filter DataFrame
tmdb_filtered_df = tmdb_df[['title', 'year', 'genre_ids', 'vote_average', 'vote_count', 'release_date']]


In [24]:

#tn_df
# Convert the date column to datetime
tn_df['release_date'] = pd.to_datetime(tn_df['release_date'])

# Extract the year
tn_df['year'] = tn_df['release_date'].dt.year

# Change the 'movie' column to 'title'
tn_df.rename(columns={'movie': 'title'}, inplace=True)

# Ensure the columns are strings
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].astype(str)
tn_df['domestic_gross'] = tn_df['domestic_gross'].astype(str)

# Remove dollar signs and commas
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].str.replace('$', '').str.replace(',', '')
tn_df['domestic_gross'] = tn_df['domestic_gross'].str.replace('$', '').str.replace(',', '')

# Convert the cleaned columns to numeric
tn_df['worldwide_gross'] = pd.to_numeric(tn_df['worldwide_gross'])
tn_df['domestic_gross'] = pd.to_numeric(tn_df['domestic_gross'])

# Create 'foreign_gross' column
tn_df['foreign_gross'] = tn_df['worldwide_gross'] - tn_df['domestic_gross']

# Filter DataFrame
tn_filtered_df = tn_df[['title', 'year', 'domestic_gross', 'foreign_gross', 'release_date']]


In [25]:

# IMDB
imdb_df.rename(columns={'primary_title': 'title', 'start_year' : 'year'}, inplace=True)
imdb_filtered_df = imdb_df

imdb_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            146144 non-null  object 
 1   title               146144 non-null  object 
 2   year                146144 non-null  int64  
 3   runtime_minutes     114405 non-null  float64
 4   genres              140736 non-null  object 
 5   actors_list         99391 non-null   object 
 6   directors_list      128436 non-null  object 
 7   screenwriters_list  7820 non-null    object 
 8   averagerating       73856 non-null   float64
 9   numvotes            73856 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 11.2+ MB
